In [1]:
import requests
import json
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

from datetime import datetime
import time

# ----- User Settings 

coins=['BTC']  # Only BTC is available at cryptocompare

collectDataFrom = pd.Timestamp('2012-02-01')


cryptocompare_api_key='777db0605f662ba6667a0c8d385d317262bd221764d4a3b01dc7a629e9592a49'
requestUrls = ['https://min-api.cryptocompare.com/data/blockchain/balancedistribution/histo/day?fsym={}&api_key={}&toTs={}&limit={}',
              'https://min-api.cryptocompare.com/data/v2/histoday?fsym={}&tsym=USD&api_key={}&toTs={}&limit={}']

# ----- User Settings -- end

limit=2000
coin_dfs=[]
for coin in coins:
    time_segments = []
    timestamp = pd.Timestamp.now()
    while collectDataFrom <= timestamp : 
        segment_dfs = []
        for requestUrl in requestUrls:
            segment_df = pd.DataFrame()
            requestUrl = requestUrl.format(coin, cryptocompare_api_key, timestamp.timestamp(),limit)
            print("Request : " + requestUrl)    
            response = requests.get(requestUrl).text 
            parsed = json.loads(response)
            # print("Json : \n\n" + response)
            segment_df = pd.DataFrame(parsed['Data']['Data'])
            segment_df['time'] = pd.to_datetime(segment_df['time'], unit='s')
            segment_df.set_index(['time'],inplace=True, verify_integrity = True, drop = True)
            segment_df.sort_index(inplace=True)
            segment_dfs.append(segment_df)
        
        #segment_df.drop_duplicates(inplace=True)
        concated_segment_dfs= pd.concat(segment_dfs,axis=1)
        time_segments.append(concated_segment_dfs) 
        
        timestamp = segment_df.first_valid_index() - pd.Timedelta(1, "d") # -1 day    
        concated_segment_dfs.drop(columns=['volumefrom','volumeto','conversionType'
                                           ,'conversionSymbol', 'open','high','low'
                                           ,'id','symbol','partner_symbol'], inplace=True)
        #print("seg tail {}".format(concated_segment_dfs.tail(1)))

    df = pd.concat(time_segments, axis=0)
    df.sort_index(inplace=True)
    # print("Df {}\n".format(df))    
    df.dropna(inplace=True)
    df_wallet_ranges = [] 
    for i in range(10):
        df_wallet_range = pd.DataFrame()
        df_wallet_range['addressesCount'] = df['balance_distribution'].map(lambda x: x[i]['addressesCount'])
        df_wallet_range['totalVolume'] = df['balance_distribution'].map(lambda x: x[i]['totalVolume'])
        vindex= df.first_valid_index()  
        range_name = "{}-{}".format(df.loc[vindex,['balance_distribution']][0][i]['from'], df.loc[vindex,['balance_distribution']][0][i]['to'])
        df_wallet_range =  pd.concat({range_name: df_wallet_range}, axis=1, names=["wallet_range", "metrics"]) 
        df_wallet_ranges.append(df_wallet_range)
    range_name = "all"
    df_wallet_range = pd.DataFrame()
    df_wallet_range['price'] = df['close']
    df_wallet_range = pd.concat({range_name: df_wallet_range}, axis=1, names=["wallet_range", "metrics"]) 
    df_wallet_ranges.append(df_wallet_range)
    
    df = pd.concat(df_wallet_ranges, axis=1)
 
    df = pd.concat({coin: df}, axis=1, names=["coin", "wallet_range", "metrics"])
    coin_dfs.append(df)
df=pd.concat(coin_dfs, axis=1)
df=df.loc[collectDataFrom:]
df.sort_index(inplace=True)
df

Request : https://min-api.cryptocompare.com/data/blockchain/balancedistribution/histo/day?fsym=BTC&api_key=777db0605f662ba6667a0c8d385d317262bd221764d4a3b01dc7a629e9592a49&toTs=1677620409.473055&limit=2000


MemoryError: Cannot allocate write+execute memory for ffi.callback(). You might be running on a system that prevents this. For more information, see https://cffi.readthedocs.io/en/latest/using.html#callbacks

In [ ]:
df_aux = pd.DataFrame()
#df_aux2 = pd.DataFrame()
coin = 'BTC'

aggregate_wallets=['0.1-1','1-10', '10-100','100-1000', '1000-10000', '10000-100000', '100000-0']


addressCounts = [df[coin][w_range]['addressesCount'] for w_range in aggregate_wallets]
df_aux['addressesCount'] = sum(addressCounts)
df.loc(axis=1)[(coin,'0.1-0','addressesCount')] = df_aux['addressesCount']


aggregate_wallets.append('0.01-0.1')
addressCounts = [df[coin][w_range]['addressesCount'] for w_range in aggregate_wallets]
df_aux['addressesCount'] = sum(addressCounts)
df.loc(axis=1)[(coin,'0.01-0','addressesCount')] = df_aux['addressesCount']


aggregate_wallets.append('0.001-0.01')
addressCounts = [df[coin][w_range]['addressesCount'] for w_range in aggregate_wallets]
df_aux['addressesCount'] = sum(addressCounts)
df.loc(axis=1)[(coin,'0.001-0','addressesCount')] = df_aux['addressesCount']
df.drop(columns=['1e-08-0.001',*aggregate_wallets],inplace=True,level= "wallet_range")


df

In [ ]:
def convert_to_m2(df,dt):
    iloc_idx = df.index.get_indexer([dt], method='nearest')  # returns absolute index into df e.g. array([5])
    loc_idx = df.index[iloc_idx]                             # if you want named index
    # my_val = df.iloc[iloc_idx]
    return df.loc[loc_idx,'WM2NS'].values[0] 
df_m2 = pd.read_csv('data/WM2NS.csv')
df_m2['DATE']= pd.to_datetime(df_m2['DATE'])
df_m2.set_index(['DATE'],inplace=True, verify_integrity = True, drop = True)
df.loc(axis=1)[('USD','all', 'm2_supply')] = df.index.map(lambda dt: convert_to_m2(df_m2,dt))

btc_birthday = pd.to_datetime('2009-01-03')
m2_supply_at_genesis = convert_to_m2(df_m2,btc_birthday)
df.loc(axis=1)[('USD','all', 'm2_supply_index')] = df.loc(axis=1)[('USD','all', 'm2_supply')]/m2_supply_at_genesis
df.loc(axis=1)[('BTC','all', 'm2_adj_price')] = df.loc(axis=1)[('BTC','all', 'price')] /df.loc(axis=1)[('USD','all', 'm2_supply_index')]
df


In [ ]:
df.loc(axis=1)[(coin,'0.1-0','k')] = 21000000*df.loc(axis=1)[(coin,'all','m2_adj_price')] / ( df.loc(axis=1)[(coin,'0.1-0','addressesCount')]
                                                                              * df.loc(axis=1)[(coin,'0.1-0','addressesCount')])

df.loc(axis=1)[(coin,'0.01-0','k')] = 21000000*df.loc(axis=1)[(coin,'all','m2_adj_price')] / ( df.loc(axis=1)[(coin,'0.01-0','addressesCount')]
                                                                              * df.loc(axis=1)[(coin,'0.01-0','addressesCount')])
df.loc(axis=1)[(coin,'0.001-0','k')] = 21000000*df.loc(axis=1)[(coin,'all','m2_adj_price')] / ( df.loc(axis=1)[(coin,'0.001-0','addressesCount')]
                                                                              * df.loc(axis=1)[(coin,'0.001-0','addressesCount')])


df.loc(axis=1)[(coin,'0.001-0', 'k-sma200w')] = df.loc(axis=1)[(coin,'0.001-0','k')].rolling(200*7).mean()
df.loc(axis=1)[(coin,'0.01-0', 'k-sma200w')] = df.loc(axis=1)[(coin,'0.01-0','k')].rolling(200*7).mean()
df.loc(axis=1)[(coin,'0.1-0', 'k-sma200w')] = df.loc(axis=1)[(coin,'0.1-0','k')].rolling(200*7).mean()


df.loc(axis=1)[(coin,'0.001-0', 'k-sma50w')] = df.loc(axis=1)[(coin,'0.001-0','k')].rolling(50*7).mean()
df.loc(axis=1)[(coin,'0.01-0', 'k-sma50w')] = df.loc(axis=1)[(coin,'0.01-0','k')].rolling(50*7).mean()
df.loc(axis=1)[(coin,'0.1-0', 'k-sma50w')] = df.loc(axis=1)[(coin,'0.1-0','k')].rolling(50*7).mean()


df.loc(axis=1)[(coin,'0.001-0', 'k-sma21w')] = df.loc(axis=1)[(coin,'0.001-0','k')].rolling(21*7).mean()
df.loc(axis=1)[(coin,'0.01-0', 'k-sma21w')] = df.loc(axis=1)[(coin,'0.01-0','k')].rolling(21*7).mean()
df.loc(axis=1)[(coin,'0.1-0', 'k-sma21w')] = df.loc(axis=1)[(coin,'0.1-0','k')].rolling(21*7).mean()


df.loc(axis=1)[(coin,'0.001-0', 'k-sma90d')] = df.loc(axis=1)[(coin,'0.001-0','k')].rolling(90).mean()
df.loc(axis=1)[(coin,'0.01-0', 'k-sma90d')] = df.loc(axis=1)[(coin,'0.01-0','k')].rolling(90).mean()
df.loc(axis=1)[(coin,'0.1-0', 'k-sma90d')] = df.loc(axis=1)[(coin,'0.1-0','k')].rolling(90).mean()


df.loc(axis=1)[(coin,'0.001-0', 'k-sma30d')] = df.loc(axis=1)[(coin,'0.001-0','k')].rolling(30).mean()
df.loc(axis=1)[(coin,'0.01-0', 'k-sma30d')] = df.loc(axis=1)[(coin,'0.01-0','k')].rolling(30).mean()
df.loc(axis=1)[(coin,'0.1-0', 'k-sma30d')] = df.loc(axis=1)[(coin,'0.1-0','k')].rolling(30).mean()

# df=df.loc['2013-01-01':]

In [ ]:
# Apply the default theme
import matplotlib.pyplot as plot
import seaborn as sns
coin='BTC'
sns.set_theme( palette="pastel")
sns.set(rc = {'figure.figsize':(16,8)})

plot.title("{} Wallet Count".format(coin), fontsize=20)

g_results=sns.lineplot(data=df[coin], x=df[coin].index, y=df[coin]['0.001-0']['addressesCount'], color='red',label='Wallets over 0.001 {}'.format(coin))
g_results=sns.lineplot(data=df[coin], x=df[coin].index, y=df[coin]['0.01-0']['addressesCount'], color='orange',label='Wallets over 0.01 {}'.format(coin))

#g_results=sns.lineplot(data=df[coin], x=df[coin].index, y=df[coin]['0.001-0.01']['addressesCount'], color='blue',label='0.001-0.01 Wallets')
#g_results=sns.lineplot(data=df[coin], x=df[coin].index, y=df[coin]['0.01-0.1']['addressesCount'], color='green',label='0.01-0.1 Wallets')
#g_results=sns.lineplot(data=df[coin], x=df[coin].index, y=df[coin]['0.1-1']['addressesCount'], color='red',label='0.1-1 Wallets')


#g_results=sns.lineplot(data=df[coin], x=df[coin].index, y=df[coin][buyAbove], color='red', label=buyAbove)
#g_results=sns.lineplot(data=df[coin], x=df[coin].index, y=df[coin]['trade_usd_value'], color='blue', label='Trading Value {}'.format(buyAbove),linewidth=1)

#g_results.set(yscale='log')
#plot.legend()

In [ ]:
#plot.title("{} k".format(coin), fontsize=20)
fig, ax = plot.subplots()

plot.title("Metcalfe's k constant for {}".format(coin), fontsize=20)

ax.set_ylim(0, 0.0018)
#g_results=sns.lineplot(data=df[coin], x=df[coin].index, y=df[coin]['0.001-0']['addressesCount'], color='red',label='Wallets over 0.001 {}'.format(coin))
#g_results=sns.lineplot(data=df[coin], x=df[coin].index, y=df[coin]['0.01-0']['k-sma30d'], color='orange',label='Metcalfe\'s k constant for wallets over 0.01 {}'.format(coin))
g_results=sns.lineplot(data=df[coin], x=df[coin].index, y=df[coin]['0.001-0']['k-sma30d'], color='red',label='Metcalfe\'s k constant for wallets over 0.001 {}'.format(coin))
#g_results.set(yscale='log')
plot.legend(loc='upper left')
ax2 = plot.twinx()
ax2.grid(False)

g_results=sns.lineplot(ax=ax2, data=df[coin], x=df[coin].index, y=df[coin]['all']['m2_adj_price'], linewidth=1, color='blue',label='USD M2 Supply adjusted {} Price'.format(coin))
x = ax2.legend(loc='upper right')


In [ ]:
#plot.title("{} k".format(coin), fontsize=20)
fig, ax = plot.subplots()

plot.title("Metcalfe's k constant for {}".format(coin), fontsize=20)

ax.set_ylim(0, 0.008)
g_results=sns.lineplot(data=df[coin], x=df[coin].index, y=df[coin]['0.01-0']['k-sma30d'], color='orange',label='Metcalfe\'s k constant for wallets over 0.01 {}'.format(coin))
#g_results=sns.lineplot(data=df[coin], x=df[coin].index, y=df[coin]['0.001-0']['k-sma30d'], color='red',label='Metcalfe\'s k constant for wallets over 0.001 {}'.format(coin))
#g_results.set(yscale='log')
plot.legend(loc='upper left')
ax2 = plot.twinx()
ax2.grid(False)

g_results=sns.lineplot(ax=ax2, data=df[coin], x=df[coin].index, y=df[coin]['all']['m2_adj_price'], linewidth=1, color='blue',label='USD M2 Supply adjusted {} Price'.format(coin))
x = ax2.legend(loc='upper right')


In [ ]:
#plot.title("{} k".format(coin), fontsize=20)
fig, ax = plot.subplots()

plot.title("Metcalfe's k constant for {}".format(coin), fontsize=20)

ax.set_ylim(0, 0.06)
g_results=sns.lineplot(data=df[coin], x=df[coin].index, y=df[coin]['0.1-0']['k-sma30d'], color='green',label='Metcalfe\'s k constant for wallets over 0.1 {}'.format(coin))
#g_results.set(yscale='log')
plot.legend(loc='upper left')
ax2 = plot.twinx()
ax2.grid(False)

g_results=sns.lineplot(ax=ax2, data=df[coin], x=df[coin].index, y=df[coin]['all']['m2_adj_price'], linewidth=1, color='blue',label='USD M2 Supply adjusted {} Price'.format(coin))
x = ax2.legend(loc='upper right')
